In [32]:
import pandas as pd
import json
import datetime as dt
import sidetable as stb

In [33]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 50

# Limpieza

In [34]:
# cargamos los datos

with open('../datos/clima.json', 'r') as f:
    clima_json = json.load(f)

In [35]:
clima_json.keys()

dict_keys(['Elche', 'Ottawa', 'Adelaide', 'Helsinki'])

Los datos están en un diccionario donde los keys son las ciudades, y los values son listas de strings. Cada string contiene los datos de un mes.

In [36]:
def crear_dataframes(json):
    
    dicc_df = {}
    
    # iteramos por el diccionario contenido en el json
    for k, v in json.items():
        
        dicc_df[k] = []
        
        lista_df = []
        lista_faltas = []
        año = 2001
        
        # iteramos por cada string contenido en la lista de valores
        # cada elemento es un string de los datos de un mes
        for i in v:
            # dividimos los datos por los saltos de linea
            datos = i.split('\n')

            # definimos los nombres de las columnas (del dataframe que se va a crear) como la primera string de la nueva lista de datos
            columnas = datos.pop(0).split(')')
            for indice, columna in enumerate(columnas):
                columnas[indice] = columna.strip().replace(' ', '_') + ')'
            columnas.pop(-1)
            
            # creamos un diccionario donde los keys son los nombres de las columnas 
            dicc_datos = {}
            dicc_datos['dias'] = []
            for col in columnas:
                dicc_datos[col] = []
            
            mes = datos[0]
            if mes == "Jan":
                año += 1
                   
            if len(datos) < 203:
                lista_faltas.append((k, año, mes))

                pass
            else:
                # iteramos por una lista de posibles valores de la longitud de un mes
                # usamos este valor para sacar los datos segun los indices
                for n in range(27,34):
                    if datos[n] == 'Max Avg Min':
                        for x, key in enumerate(dicc_datos.keys()):
                            dicc_datos[key] = datos[x*n:x*n+n]
                # creamos un dataframe de los datos
                df = pd.DataFrame(dicc_datos)
                
                # creamos una copia del dataframe
                df2 = df.copy()
                for col in df2.columns[1:-1]:
                    try:
                        df2[col] = df2[col].str.split(' ')

                        # display(df2)

                    except:
                        print('error split:', k, mes, año)
                
                # dividimos las columnas que tienen datos combinados, usando la copia del dataframe
                # para sacar los nombres de las columnas 
                try:
                    for col in df.columns[1:-1]:
                        col1 = f'{col}_{df2.loc[0,col][0]}'
                        col2 = f'{col}_{df2.loc[0,col][1]}'
                        col3 = f'{col}_{df2.loc[0,col][2]}'
                        df[[col1, col2, col3]] = df[col].str.split(' ', expand=True)
                        df.drop(col, axis=1, inplace=True)
                except:
                    print('error columnas:', k, mes, año)
                    
                # eliminamos la primera fila que ahora es redundante y ponemos la columna de dias como índice    
                df.drop(0, axis=0, inplace=True)
                df['Month'] = mes
                df['Year'] = año
                df['Date'] =  df['dias'] + "-" + mes + "-" + str(año)
                # df.set_index('dias', inplace=True)

                lista_df.append(df)
                if mes == 'Dec':
                    df_año = pd.DataFrame()
                    df_año = pd.concat(lista_df, axis=0)
                    dicc_df[k].append(df_año)
                    lista_df = []
                    
    print(f"Meses que faltan: {lista_faltas}")            
    return dicc_df

In [37]:
# aplicamos la funcion al contenido del archivo de json que contiene los datos

dicc_datos = crear_dataframes(clima_json)

Meses que faltan: [('Helsinki', 2004, 'Jan')]


In [38]:
dicc_datos

{'Elche': [   dias Precipitation_(in) Time_Temperature_(°F)_Max  \
  1     1               0.00                        57   
  2     2               0.00                        57   
  3     3               0.00                        57   
  4     4               0.00                        59   
  5     5               0.00                        57   
  ..  ...                ...                       ...   
  27   27               0.00                        63   
  28   28               0.00                        64   
  29   29               0.00                        70   
  30   30               0.00                        70   
  31   31               0.00                        68   
  
     Time_Temperature_(°F)_Avg Time_Temperature_(°F)_Min Dew_Point_(°F)_Max  \
  1                       54.9                        54                 55   
  2                       54.9                        50                 52   
  3                       52.8                        4

In [39]:
# comprobamos que se han creado todos los keys del diccionario de dataframes

dicc_datos.keys()

dict_keys(['Elche', 'Ottawa', 'Adelaide', 'Helsinki'])

In [40]:
# concatenamos los dataframes de cada ciudad

for k, v in dicc_datos.items():
    dicc_datos[k] = pd.concat(v, axis=0)

In [41]:
# creamos una nueva columna de ciudad 

for k, v in dicc_datos.items():
    v['City'] = k

In [42]:
# creamos el dataframe final concatenando los cuatro dataframes de las ciudades

df_final = pd.concat([dicc_datos['Elche'], dicc_datos['Adelaide'],
                     dicc_datos['Helsinki'], dicc_datos['Ottawa']], axis=0)

In [43]:
# cambiamos el tipo de datos de la columna de fechas a tipo datetime

df_final['Date'] = df_final['Date'].apply(pd.to_datetime)

In [44]:
# comprobamos que el dataframe contiene todos los datos

df_final.shape

(30626, 21)

In [45]:
# examinamos los tipos de datos

df_final.dtypes

dias                                 object
Precipitation_(in)                   object
Time_Temperature_(°F)_Max            object
Time_Temperature_(°F)_Avg            object
Time_Temperature_(°F)_Min            object
Dew_Point_(°F)_Max                   object
Dew_Point_(°F)_Avg                   object
Dew_Point_(°F)_Min                   object
Humidity_(%)_Max                     object
Humidity_(%)_Avg                     object
Humidity_(%)_Min                     object
Wind_Speed_(mph)_Max                 object
Wind_Speed_(mph)_Avg                 object
Wind_Speed_(mph)_Min                 object
Pressure_(in)_Max                    object
Pressure_(in)_Avg                    object
Pressure_(in)_Min                    object
Month                                object
Year                                  int64
Date                         datetime64[ns]
City                                 object
dtype: object

In [46]:
# hacemos una lista de las columnas que son de tipo object que deberian ser tipo float

columnas_num = ['Precipitation_(in)', 'Time_Temperature_(°F)_Max',
       'Time_Temperature_(°F)_Avg', 'Time_Temperature_(°F)_Min',
       'Dew_Point_(°F)_Max', 'Dew_Point_(°F)_Avg', 'Dew_Point_(°F)_Min',
       'Humidity_(%)_Max', 'Humidity_(%)_Avg', 'Humidity_(%)_Min',
       'Wind_Speed_(mph)_Max', 'Wind_Speed_(mph)_Avg', 'Wind_Speed_(mph)_Min',
       'Pressure_(in)_Max', 'Pressure_(in)_Avg', 'Pressure_(in)_Min']

In [47]:
# cambiamos estas columnas a tipo float

for i in columnas_num:
    df_final[i] = df_final[i].astype('float64')

In [48]:
# funcion para convertir las temperaturas de Farenheit en Celsius

def far_to_cel(columna):
    try:
        return round(((columna - 32) * (5/9)), 1)
    except:
        print(columna)
        return columna

In [49]:
# aplicamos la funcion de conversion a las columnas que contienen datos de temperaturas

for i in ['Time_Temperature_(°F)_Max',
       'Time_Temperature_(°F)_Avg', 'Time_Temperature_(°F)_Min',
       'Dew_Point_(°F)_Max', 'Dew_Point_(°F)_Avg', 'Dew_Point_(°F)_Min']:
    
    df_final[i] = df_final[i].apply(far_to_cel)

In [50]:
# aplicamos una lambda a la columna de precipitacion para convertirla a milimetros

for i in ['Precipitation_(in)']:
    df_final[i] = df_final[i].apply(lambda x: round(x * 25.4))

In [51]:
# aplicamos una lambda a las columnas de presion atmosferica para convertirlas a hectoPascales

for i in ['Pressure_(in)_Max', 'Pressure_(in)_Avg', 'Pressure_(in)_Min']:
    df_final[i] = df_final[i].apply(lambda x: round(x * 33.86389))

In [52]:
# aplicamos una lambda para convertir las velocidades de viento en kilometros por hora

for i in ['Wind_Speed_(mph)_Max', 'Wind_Speed_(mph)_Avg', 'Wind_Speed_(mph)_Min']:
    df_final[i] = df_final[i].apply(lambda y: round(y * 1.609344, 1))

In [53]:
# imprimimos las columnas en el formato de diccionario

for col in df_final.columns:
    print(f"'{col}': '{col}', ")

'dias': 'dias', 
'Precipitation_(in)': 'Precipitation_(in)', 
'Time_Temperature_(°F)_Max': 'Time_Temperature_(°F)_Max', 
'Time_Temperature_(°F)_Avg': 'Time_Temperature_(°F)_Avg', 
'Time_Temperature_(°F)_Min': 'Time_Temperature_(°F)_Min', 
'Dew_Point_(°F)_Max': 'Dew_Point_(°F)_Max', 
'Dew_Point_(°F)_Avg': 'Dew_Point_(°F)_Avg', 
'Dew_Point_(°F)_Min': 'Dew_Point_(°F)_Min', 
'Humidity_(%)_Max': 'Humidity_(%)_Max', 
'Humidity_(%)_Avg': 'Humidity_(%)_Avg', 
'Humidity_(%)_Min': 'Humidity_(%)_Min', 
'Wind_Speed_(mph)_Max': 'Wind_Speed_(mph)_Max', 
'Wind_Speed_(mph)_Avg': 'Wind_Speed_(mph)_Avg', 
'Wind_Speed_(mph)_Min': 'Wind_Speed_(mph)_Min', 
'Pressure_(in)_Max': 'Pressure_(in)_Max', 
'Pressure_(in)_Avg': 'Pressure_(in)_Avg', 
'Pressure_(in)_Min': 'Pressure_(in)_Min', 
'Month': 'Month', 
'Year': 'Year', 
'Date': 'Date', 
'City': 'City', 


In [54]:
# renombramos las columnas para reflejar las nuevas unidades despues de las conversiones que hemos hecho

df_final.rename(columns = {'Precipitation_(in)': 'Precipitation_(mm)', 
                           'Time_Temperature_(°F)_Max': 'Temperature_(°C)_Max', 
                            'Time_Temperature_(°F)_Avg': 'Temperature_(°C)_Avg', 
                            'Time_Temperature_(°F)_Min': 'Temperature_(°C)_Min', 
                            'Dew_Point_(°F)_Max': 'Dew_Point_(°C)_Max', 
                            'Dew_Point_(°F)_Avg': 'Dew_Point_(°C)_Avg', 
                            'Dew_Point_(°F)_Min': 'Dew_Point_(°C)_Min', 
                            'Humidity_(%)_Max': 'Humidity_(%)_Max', 
                            'Humidity_(%)_Avg': 'Humidity_(%)_Avg', 
                            'Humidity_(%)_Min': 'Humidity_(%)_Min', 
                            'Wind_Speed_(mph)_Max': 'Wind_Speed_(kph)_Max', 
                            'Wind_Speed_(mph)_Avg': 'Wind_Speed_(kph)_Avg', 
                            'Wind_Speed_(mph)_Min': 'Wind_Speed_(kph)_Min', 
                            'Pressure_(in)_Max': 'Pressure_(hPa)_Max', 
                            'Pressure_(in)_Avg': 'Pressure_(hPa)_Avg', 
                            'Pressure_(in)_Min': 'Pressure_(hPa)_Min', 
                            'Month': 'Month', 
                            'Year': 'Year', 
                            'Date': 'Date', 
                            'City': 'City'}, inplace = True)

In [55]:
# cambiamos las columnas de año y mes a tipo datetime, basado en la columna fecha

df_final['Year'] = df_final['Date'].apply(lambda x: x.strftime('%Y'))
df_final['Month'] = df_final['Date'].apply(lambda x: x.strftime('%B'))

In [56]:
# creamos una funcion para crear una columna de estaciones

def estacion(x, ciudad):
    if ciudad == 'Adelaide':
        if x.strftime('%m-%d') <= '03-20':
            return 'summer'
        elif x.strftime('%m-%d') <= '06-20':
            return 'autumn'
        elif x.strftime('%m-%d') <= '09-20' :
            return 'winter'
        elif x.strftime('%m-%d') <= '12-20':
            return 'spring'
        elif x.strftime('%m-%d') <= '12-31':
            return 'summer'
    else:
        if x.strftime('%m-%d') <= '03-20':
            return 'winter'
        elif x.strftime('%m-%d') <= '06-20':
            return 'spring'
        elif x.strftime('%m-%d') <= '09-20' :
            return 'summer'
        elif x.strftime('%m-%d') <= '12-20':
            return 'autumn'
        elif x.strftime('%m-%d') <= '12-31':
            return 'winter'

In [57]:
# aplicamos la funcion de estaciones

df_final['Season'] = df_final.apply(lambda x: estacion(x['Date'], x['City']), axis=1)

In [58]:
df_final['Latitude'] = df_final['City'].map({'Ottawa':45.3202, 'Helsinki': 60.3179, 'Elche': 38.2851, 'Adelaide': -34.9462})
df_final['Longitude'] = df_final['City'].map({'Ottawa':-75.6682, 'Helsinki': 24.9496, 'Elche': -0.5625, 'Adelaide': 138.5312})

In [59]:
# reordenamos las columnas a una orden mas logica

reorder = ['City', 'Latitude', 'Longitude', 'Date', 'Month', 'Year', 'Season', 'Precipitation_(mm)', 'Temperature_(°C)_Max',
       'Temperature_(°C)_Avg', 'Temperature_(°C)_Min', 'Dew_Point_(°C)_Max',
       'Dew_Point_(°C)_Avg', 'Dew_Point_(°C)_Min', 'Humidity_(%)_Max',
       'Humidity_(%)_Avg', 'Humidity_(%)_Min', 'Wind_Speed_(kph)_Max',
       'Wind_Speed_(kph)_Avg', 'Wind_Speed_(kph)_Min', 'Pressure_(hPa)_Max',
       'Pressure_(hPa)_Avg', 'Pressure_(hPa)_Min']

df_final = df_final.reindex(columns = reorder)

In [60]:
# examinamos el dataframe final

df_final.head()

,City,Latitude,Longitude,Date,Month,Year,Season,Precipitation_(mm),Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,Dew_Point_(°C)_Max,Dew_Point_(°C)_Avg,Dew_Point_(°C)_Min,Humidity_(%)_Max,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(kph)_Max,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min
1,Elche,38.2851,-0.5625,2002-01-01,January,2002,winter,0,13.9,12.7,12.2,12.8,8.5,5.0,94.0,75.9,62.0,37.0,25.1,19.3,1023,1023,1019
2,Elche,38.2851,-0.5625,2002-01-02,January,2002,winter,0,13.9,12.7,10.0,11.1,8.8,7.2,82.0,76.7,67.0,40.2,23.0,4.8,1023,1019,1019
3,Elche,38.2851,-0.5625,2002-01-03,January,2002,winter,0,13.9,11.6,8.9,11.1,8.8,7.2,94.0,83.8,72.0,14.5,5.6,0.0,1019,1016,1013
4,Elche,38.2851,-0.5625,2002-01-04,January,2002,winter,0,15.0,12.5,8.9,11.1,8.7,6.1,94.0,77.6,63.0,29.0,12.9,3.2,1023,1016,1013
5,Elche,38.2851,-0.5625,2002-01-05,January,2002,winter,0,13.9,13.0,12.2,11.1,7.3,5.0,88.0,68.2,58.0,27.4,23.3,9.7,1029,1026,1023


In [61]:
df_final.shape

(30626, 23)

In [62]:
# guardamos los datos en un csv

df_final.to_csv('../datos/clima_2002-2022.csv')

In [63]:
# guardamos los datos en un archivo pickle

df_final.to_pickle('../datos/clima_2002-2022.pkl')